In [252]:
import sqlite3
import pandas as pd

In [253]:
# Connect to the SQLite database
con = sqlite3.connect("Topic1_dataset.sqlite")

# Create a cursor to execute SQL commands
cur = con.cursor()

In [254]:
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cur.fetchall()
tables

[('sqlite_sequence',), ('products',), ('purchase',), ('sales',), ('shops',)]

In [255]:
def get_pd_dataframe(query, connection):
    return pd.read_sql_query(query, connection)

products_preview = get_pd_dataframe("SELECT * FROM products ;", con)

purchase_preview = get_pd_dataframe("SELECT * FROM purchase ;", con)

shops_preview = get_pd_dataframe("SELECT * FROM shops;", con)

sales_preview = get_pd_dataframe("SELECT * FROM sales;", con)


In [257]:
zeroprice = products_preview[products_preview['price']== 0]
zeroprice


,product,title,price,book_original_price,barcode,barcode2,isbn
1710,9781546176183,DOG MAN #14: Big Jim Believes,0.000000,159.90,9781546176183,9781546176183,9781546176183
2302,PREORDER,Pre-order: Dog Man #9 Crime and Punishment (97...,0.000000,None,000640,000640,SPECIAL PRICE
2303,PREORDER-01,Pre-order: 130 Storey Treehouse (9781529017922),0.000000,89.00,None,000719,None
2304,PREORDER-03,Pre-order: Dog Man #10 Mothering Heights (9781...,0.000000,None,000728,000728,None


In [258]:
# Find rows where barcode2 is duplicated
duplicates = products_preview[products_preview['barcode2'].duplicated(keep=False)]

# Display all columns for these rows
duplicates


,product,title,price,book_original_price,barcode,barcode2,isbn
530,1419101114,Paw Prints B.Bks: Numbers,9.900000,9.90,9781419101113,786943084221,None
531,1419101122,Paw Prints B.Bks: Best Friends,9.900000,9.90,9781419101120,786943084221,None
532,1419101130,Paw Prints B.Bks: Colors,9.900000,9.90,9781419101137,786943084221,None
533,1419402161,All About Me! Foam Bk: S.St. - Big Bird,35.000000,35.00,9781419402166,786943086102,None
534,141940217X,All About Me! Foam Bk: S.St. - Cookie Monster,35.000000,35.00,9781419402173,786943086102,None
535,1419402188,All About Me! Foam Bk: S.St. - Elmo,35.000000,35.00,9781419402180,786943086102,None
694,1576571742,Super Chunky! First Words B.Bks: ABC,9.900000,9.90,9781576571743,786943004793,None
695,1576571750,Super Chunky! First Words B.Bks: Helping Hands,9.900000,9.90,9781576571750,786943004793,None
696,1576571769,Super Chunky! First Words B.Bks: Welcome to My...,9.900000,9.90,9781576571767,786943004793,None
697,1576571777,"Super Chunky! First Words B.Bks: Beep, Vroom, ...",9.900000,9.90,9781576571774,786943004793,None


In [259]:
# Merge sales_preview with shops_preview and products_preview
merged_df = sales_preview.merge(shops_preview[['location', 'shopname']], on='location', how='left')
merged_df = merged_df.merge(products_preview[['product', 'title', 'price','barcode2']].rename(columns={'price': 'market_price'}), on='product', how='left')

In [260]:
# Calculate final price
merged_df['final_price']= merged_df['amount'] / merged_df['quantity']

In [261]:
# Calculate final discount percentage
merged_df['final_discount'] = (merged_df['market_price'] - merged_df['final_price'] ) / merged_df['market_price'] * 100

In [262]:
# Change channel to 'Bookfair - Roadshow' if client name is 'Hong Kong Book Fair'
merged_df.loc[merged_df['clients'] == 'Hong Kong Book Fair', 'channel'] = 'Bookfair - Roadshow'

# Verify the update
print(merged_df[merged_df['clients'] == 'Hong Kong Book Fair'][['clients', 'channel']].head())

                   clients              channel
47833  Hong Kong Book Fair  Bookfair - Roadshow
47834  Hong Kong Book Fair  Bookfair - Roadshow
47835  Hong Kong Book Fair  Bookfair - Roadshow
47836  Hong Kong Book Fair  Bookfair - Roadshow
47837  Hong Kong Book Fair  Bookfair - Roadshow


In [263]:
# Split the merged_df into two DataFrames based on channel
merged_retail_df = merged_df[merged_df['channel'] == 'Retail']
merged_roadshow_df = merged_df[merged_df['channel'] == 'Bookfair - Roadshow']

In [264]:
# Calculate the first sale date for each product
merged_retail_df['First_sale'] = merged_retail_df.groupby('product')['trandate'].transform('min')
merged_roadshow_df['First_sale'] = merged_roadshow_df.groupby('product')['trandate'].transform('min')


In [266]:
def summarize_sales(df: pd.DataFrame, output_filename: str = None) -> pd.DataFrame:
    """
    Summarize sales data by product barcode.

    Parameters:
        df (pd.DataFrame): Input DataFrame with columns ['barcode2', 'trandate', 'First_sale', 'quantity', 'final_discount'].
        output_filename (str, optional): If provided, the summary DataFrame will be exported to this CSV file.

    Returns:
        pd.DataFrame: Summary DataFrame.
    """
    # Ensure datetime columns
    df['trandate'] = pd.to_datetime(df['trandate'])
    df['First_sale'] = pd.to_datetime(df['First_sale'])

    summary_list = []

    for product, group in df.groupby('barcode2'):
        first_sale_date = group['First_sale'].iloc[0]

        # Calculate quarter and year-quarter
        first_sale_quarter = first_sale_date.quarter
        first_sale_year_quarter = f"{first_sale_date.year}-Q{first_sale_quarter}"

        # Weekly windows
        week_1_end = first_sale_date + pd.Timedelta(weeks=1)
        week_2_end = first_sale_date + pd.Timedelta(weeks=2)
        week_4_end = first_sale_date + pd.Timedelta(weeks=4)

        def qty_until(end_date):
            return group.loc[group['trandate'] <= end_date, 'quantity'].sum()

        first_week_qty = qty_until(week_1_end)
        first_2weeks_qty = qty_until(week_2_end)
        first_4weeks_qty = qty_until(week_4_end)

        # Months since first sale
        group['months_since_first'] = (
            (group['trandate'].dt.year - first_sale_date.year) * 12 +
            (group['trandate'].dt.month - first_sale_date.month) + 1
        )

        qty_1_3 = group.loc[(group['months_since_first'] >= 1) & (group['months_since_first'] <= 3), 'quantity'].sum()
        qty_4_6 = group.loc[(group['months_since_first'] >= 4) & (group['months_since_first'] <= 6), 'quantity'].sum()
        qty_7_9 = group.loc[(group['months_since_first'] >= 7) & (group['months_since_first'] <= 9), 'quantity'].sum()
        qty_10_12 = group.loc[(group['months_since_first'] >= 10) & (group['months_since_first'] <= 12), 'quantity'].sum()

        avg_discount_12m = group.loc[group['months_since_first'] <= 12, 'final_discount'].mean() if not group.empty else 0

        summary_list.append({
            'barcode2': product,
            'First_sale': first_sale_date,
            'First_sale_quarter': first_sale_quarter,
            'First_sale_year_quarter': first_sale_year_quarter,
            'first_week_qty': first_week_qty,
            'first_2weeks_qty': first_2weeks_qty,
            'first_4weeks_qty': first_4weeks_qty,
            'avg_discount_12_months': avg_discount_12m,
            'qty_1_3_months': qty_1_3,
            'qty_4_6_months': qty_4_6,
            'qty_7_9_months': qty_7_9,
            'qty_10_12_months': qty_10_12,
        })

    summary_df = pd.DataFrame(summary_list)

    # Export if filename provided
    if output_filename:
        summary_df.to_csv(output_filename, index=False)
        print(f"Exported to {output_filename}")

    return summary_df


In [267]:
summarize_sales_retail_df = summarize_sales(merged_retail_df, output_filename="summarize_sales_retail.csv")
summarize_sales_roadshow_df = summarize_sales(merged_roadshow_df, output_filename="summarize_sales_roadshow.csv")

Exported to summarize_sales_retail.csv
Exported to summarize_sales_roadshow.csv


In [268]:
#Sample Checking for Guinness World Records 2024,9781913484385
summarize_sales_retail_df[summarize_sales_retail_df['barcode2'] == '9781913484385']

,barcode2,First_sale,First_sale_quarter,First_sale_year_quarter,first_week_qty,first_2weeks_qty,first_4weeks_qty,avg_discount_12_months,qty_1_3_months,qty_4_6_months,qty_7_9_months,qty_10_12_months
1807,9781913484385,2023-10-19,4,2023-Q4,15,26,49,4.641028,171,77,32,1


In [269]:
summarize_sales_roadshow_df[summarize_sales_roadshow_df['barcode2'] == '9781913484385']

,barcode2,First_sale,First_sale_quarter,First_sale_year_quarter,first_week_qty,first_2weeks_qty,first_4weeks_qty,avg_discount_12_months,qty_1_3_months,qty_4_6_months,qty_7_9_months,qty_10_12_months
1128,9781913484385,2023-12-17,4,2023-Q4,1,1,1,-28.870293,1,0,0,0


In [270]:
#Sample Checking for Dav Pilkey - Cat Kid Comic Club #1, 978981495800
summarize_sales_retail_df[summarize_sales_retail_df['barcode2'] == '978981495800']

,barcode2,First_sale,First_sale_quarter,First_sale_year_quarter,first_week_qty,first_2weeks_qty,first_4weeks_qty,avg_discount_12_months,qty_1_3_months,qty_4_6_months,qty_7_9_months,qty_10_12_months
1964,978981495800,2022-03-30,1,2022-Q1,14,19,28,5.794234,30,51,36,5


In [271]:
summarize_sales_roadshow_df[summarize_sales_roadshow_df['barcode2'] == '978981495800']

,barcode2,First_sale,First_sale_quarter,First_sale_year_quarter,first_week_qty,first_2weeks_qty,first_4weeks_qty,avg_discount_12_months,qty_1_3_months,qty_4_6_months,qty_7_9_months,qty_10_12_months
1272,978981495800,2022-07-26,3,2022-Q3,113,113,113,28.309232,113,0,0,0


In [272]:
def expand_quarterly_sales(df: pd.DataFrame, output_filename: str = None) -> pd.DataFrame:
    """
    Expand quarterly sales data for each product, including projections.

    Parameters:
        df (pd.DataFrame): Input DataFrame with columns ['barcode2', 'trandate', 'quantity', 'final_discount'].
        output_filename (str, optional): If provided, the expanded DataFrame will be exported to this CSV file.

    Returns:
        pd.DataFrame: Expanded quarterly summary DataFrame.
    """
    # Ensure datetime and quarter columns
    df['trandate'] = pd.to_datetime(df['trandate'])
    df['year_quarter'] = df['trandate'].dt.to_period('Q')

    expanded_rows = []

    for product, group in df.groupby('barcode2'):
        first_sale_date = group['trandate'].min()
        quarterly_sales = group.groupby('year_quarter')['quantity'].sum()
        quarterly_discount = group.groupby('year_quarter')['final_discount'].mean()

        # Full range of quarters from first to last
        start_q = quarterly_sales.index.min()
        end_q = quarterly_sales.index.max()
        full_quarters = pd.period_range(start=start_q, end=end_q, freq='Q')

        # Reindex to include missing quarters
        quarterly_sales = quarterly_sales.reindex(full_quarters, fill_value=0)
        quarterly_discount = quarterly_discount.reindex(full_quarters)

        for idx, current_q in enumerate(full_quarters):
            prev_qty = quarterly_sales.iloc[idx - 1] if idx > 0 else 0
            curr_qty = quarterly_sales.iloc[idx]
            avg_discount = quarterly_discount.iloc[idx]

            # Quarter number (1–4)
            quarter_num = current_q.quarter

            # Projections: next 4 quarters after current_q
            projections = []
            for i in range(1, 5):
                next_q = current_q + i
                projections.append(quarterly_sales.get(next_q, pd.NA))

            expanded_rows.append({
                'barcode2': product,
                'Current_quarter': str(current_q),
                'Quarter_num': quarter_num,
                'Previous_quarter_qty': prev_qty,
                'Current_quarter_qty': curr_qty,
                'Avg_discount': avg_discount,
                'First_day': first_sale_date,
                'Next_Q1': projections[0],
                'Next_Q2': projections[1],
                'Next_Q3': projections[2],
                'Next_Q4': projections[3]
            })

    quarter_expanded_df = pd.DataFrame(expanded_rows)

    # Export if filename provided
    if output_filename:
        quarter_expanded_df.to_csv(output_filename, index=False)
        print(f"Exported to {output_filename}")

    return quarter_expanded_df


In [273]:
expand_quarterly_sales_retail_df = expand_quarterly_sales(merged_retail_df, output_filename="expand_quarterly_sales_retail.csv")
expand_quarterly_sales_roadshow_df = expand_quarterly_sales(merged_roadshow_df, output_filename="expand_quarterly_sales_roadshow.csv")

Exported to expand_quarterly_sales_retail.csv
Exported to expand_quarterly_sales_roadshow.csv


In [274]:
#Sample Checking for Guinness World Records 2024,9781913484385
expand_quarterly_sales_retail_df[expand_quarterly_sales_retail_df['barcode2'] == '9781913484385']

,barcode2,Current_quarter,Quarter_num,Previous_quarter_qty,Current_quarter_qty,Avg_discount,First_day,Next_Q1,Next_Q2,Next_Q3,Next_Q4
48042,9781913484385,2023Q4,4,0,171,0.601308,2023-10-19,77,32,1,<NA>
48043,9781913484385,2024Q1,1,171,77,9.936967,2023-10-19,32,1,<NA>,<NA>
48044,9781913484385,2024Q2,2,77,32,13.125000,2023-10-19,1,<NA>,<NA>,<NA>
48045,9781913484385,2024Q3,3,32,1,0.000000,2023-10-19,<NA>,<NA>,<NA>,<NA>


In [275]:
expand_quarterly_sales_roadshow_df[expand_quarterly_sales_roadshow_df['barcode2'] == '9781913484385']


,barcode2,Current_quarter,Quarter_num,Previous_quarter_qty,Current_quarter_qty,Avg_discount,First_day,Next_Q1,Next_Q2,Next_Q3,Next_Q4
17391,9781913484385,2023Q4,4,0,1,-28.870293,2023-12-17,<NA>,<NA>,<NA>,<NA>


In [281]:
#Sample Checking for Dav Pilkey - Cat Kid Comic Club #1, 978981495800
expand_quarterly_sales_retail_df[expand_quarterly_sales_retail_df['barcode2'] == '978981495800']

,barcode2,Current_quarter,Quarter_num,Previous_quarter_qty,Current_quarter_qty,Avg_discount,First_day,Next_Q1,Next_Q2,Next_Q3,Next_Q4
52049,978981495800,2022Q1,1,0,3,20.022247,2022-03-30,46,47,26,0
52050,978981495800,2022Q2,2,3,46,8.076607,2022-03-30,47,26,0,15
52051,978981495800,2022Q3,3,46,47,2.130026,2022-03-30,26,0,15,19
52052,978981495800,2022Q4,4,47,26,6.738256,2022-03-30,0,15,19,8
52053,978981495800,2023Q1,1,26,0,NaN,2022-03-30,15,19,8,5
52054,978981495800,2023Q2,2,0,15,1.334816,2022-03-30,19,8,5,13
52055,978981495800,2023Q3,3,15,19,0.526901,2022-03-30,8,5,13,17
52056,978981495800,2023Q4,4,19,8,0.000000,2022-03-30,5,13,17,9
52057,978981495800,2024Q1,1,8,5,0.000000,2022-03-30,13,17,9,0
52058,978981495800,2024Q2,2,5,13,0.000000,2022-03-30,17,9,0,1


In [282]:
expand_quarterly_sales_roadshow_df[expand_quarterly_sales_roadshow_df['barcode2'] == '978981495800']

,barcode2,Current_quarter,Quarter_num,Previous_quarter_qty,Current_quarter_qty,Avg_discount,First_day,Next_Q1,Next_Q2,Next_Q3,Next_Q4
20126,978981495800,2022Q3,3,0,113,28.309232,2022-07-26,0,0,0,91
20127,978981495800,2022Q4,4,113,0,NaN,2022-07-26,0,0,91,0
20128,978981495800,2023Q1,1,0,0,NaN,2022-07-26,0,91,0,0
20129,978981495800,2023Q2,2,0,0,NaN,2022-07-26,91,0,0,0
20130,978981495800,2023Q3,3,0,91,45.494994,2022-07-26,0,0,0,111
20131,978981495800,2023Q4,4,91,0,NaN,2022-07-26,0,0,111,<NA>
20132,978981495800,2024Q1,1,0,0,NaN,2022-07-26,0,111,<NA>,<NA>
20133,978981495800,2024Q2,2,0,0,NaN,2022-07-26,111,<NA>,<NA>,<NA>
20134,978981495800,2024Q3,3,0,111,45.071291,2022-07-26,<NA>,<NA>,<NA>,<NA>
